## Using Micrometry to identify the affine transform between two cameras.

In this example we are trying to determine the coefficients $A,B,C,D,E,F$ in this equation:

\begin{equation*}
\begin{bmatrix}
x_{f}\\
y_{f}
\end{bmatrix}
=
\begin{bmatrix}
A & B & C\\
D & E & F
\end{bmatrix}
\times
\begin{bmatrix}
1\\
x_{i}\\
y_{i}\\
\end{bmatrix}
\end{equation*}

This is a simple first order transformation. It is accurate enough for multiplane SMLM analysis with `storm_analysis.multi_plane.multi_plane` and `storm_analysis.multi_plane.multi_plane_dao`. If greater precision is desired it should provide a good starting point for identifying higher order transformations.

The coefficient determination could fail for images with large differences in field curvature.

Micrometry uses the geometric hashing approach described in [Lang et al, The Astronomical Journal, 2010](http://dx.doi.org/10.1088/0004-6256/139/5/1782) to determine the affine transform.


### Configuration

The typical starting point is movies of fluorescent beads taken with the two different cameras at the same time. A single (in focus) frame from each camera is then analyzed with `3D-DAOSTORM` or `sCMOS` to identify the bead locations.

In this example we're just going to generate the localizations files.


In [ ]:
import matplotlib
import matplotlib.pyplot as pyplot
import numpy
import os

import storm_analysis.sa_library.sa_h5py as saH5Py
import storm_analysis.micrometry.micrometry as micrometry

import os

# Change to working directory.
os.chdir("/home/hbabcock/Data/storm_analysis/jy_testing/")
print(os.getcwd())

### Identity transform

In [ ]:
# Create the test data.

im_size = 512
n_points = 50

numpy.random.seed(0)

locs = {"x" : numpy.random.uniform(high = im_size, size = n_points),
        "y" : numpy.random.uniform(high = im_size, size = n_points)}

#
# Note that micrometry needs to know the size of the field of view in pixels
# in order to work properly.
#
with saH5Py.SAH5Py("locs1.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs, 0)
    
with saH5Py.SAH5Py("locs2.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs, 0)

**A note on the parameters:**

* `min_size` - The minimum distance in pixels between two localizations to be considered for quad formation. You probably won't need to change this.
* `max_size` - The maximum distance in pixels between two localizations to be considered for quad formation. You may need to change this depending on localization sparsity.
* `max_neighbors` - The number of nearest neighbors to consider for quad formation. You might also need to change this.

Basically what micrometry is going to do is to build a list of quads (a quad is a group of 4 localizations) for the two data-sets, then compare the quads against each other. Increasing `max_size` and `max_neighbors` will increase the number of quads that are found and compared. However since the comparison scales as $N^2$ you are trying to balance having enough quads to find a match against having so many that it takes forever to compare them all.

In [ ]:
# Find the transform
mm = micrometry.Micrometry("locs1.hdf5",
                           min_size = 5.0,
                           max_size = 100.0,
                           max_neighbors = 20)

# 1.0e-2 is a relative unit used when comparing quads.
#
[best_ratio, best_transform] = mm.findTransform("locs2.hdf5", 1.0e-2)


Note : `best_ratio` is a metric for how much better the match is than random chance. A value above ~10.0 indicates that the match is very likely not random. The scale is logarithmic so 12.0 is much better than 10.0.

In [ ]:
# Print the ratio and plot the best match.

print("Best ratio: {0:.1f}".format(best_ratio))
print("Best transform:")
print()
micrometry.prettyPrintTransform(best_transform)

# Change default figure size.
matplotlib.rcParams['figure.figsize'] = (8,6)

micrometry.plotMatch(mm.getRefKDTree(),
                     mm.getOtherKDTree(),
                     best_transform)


### Non-identity transform

In [ ]:
locs2 = {"x" : numpy.zeros(n_points),
         "y" : numpy.zeros(n_points)}

tt = [numpy.array([-1.0, -0.9, 0.1]), 
      numpy.array([2.0, 0.2, 0.9])]

for i in range(n_points):
    locs2["x"][i] = tt[0][0] + tt[0][1] * locs["x"][i] + tt[0][2] * locs["y"][i]
    locs2["y"][i] = tt[1][0] + tt[1][1] * locs["x"][i] + tt[1][2] * locs["y"][i]

with saH5Py.SAH5Py("locs3.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs2, 0)

In [ ]:
# Plot localizations

fig = pyplot.figure(figsize = (10,5))
ax1 = fig.add_subplot(1,2,1)
ax1.scatter(locs["x"], locs["y"], s = 8)

ax2 = fig.add_subplot(1,2,2)
ax2.scatter(locs2["x"], locs2["y"], s = 8)

pyplot.show()


In [ ]:
# Find the transform
#
# Note that we increased max_size a little here in order to find the transform.
#
mm = micrometry.Micrometry("locs1.hdf5",
                           min_size = 5.0,
                           max_size = 150.0,
                           max_neighbors = 20)

[best_ratio, best_transform] = mm.findTransform("locs3.hdf5", 1.0e-2)

In [ ]:
# Print the ratio and plot the best match.

print("Best ratio: {0:.1f}".format(best_ratio))
print("Best transform:")
print()
micrometry.prettyPrintTransform(best_transform)

micrometry.plotMatch(mm.getRefKDTree(),
                     mm.getOtherKDTree(),
                     best_transform)

### Add some noise localizations

In [ ]:
n_random = 10

locs4 = {"x" : numpy.concatenate((locs["x"], numpy.random.uniform(high = im_size, size = n_random))),
         "y" : numpy.concatenate((locs["y"], numpy.random.uniform(high = im_size, size = n_random)))}

locs5 = {"x" : numpy.concatenate((locs["x"], numpy.random.uniform(high = im_size, size = n_random))),
         "y" : numpy.concatenate((locs["y"], numpy.random.uniform(high = im_size, size = n_random)))}

tt = [numpy.array([-1.0, -0.9, 0.1]), 
      numpy.array([2.0, 0.2, 0.9])]

for i in range(n_points + n_random):
    x = locs5["x"][i]
    y = locs5["y"][i]
    locs5["x"][i] = tt[0][0] + tt[0][1] * x + tt[0][2] * y
    locs5["y"][i] = tt[1][0] + tt[1][1] * x + tt[1][2] * y
    
with saH5Py.SAH5Py("locs4.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs4, 0)
    
with saH5Py.SAH5Py("locs5.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs5, 0)
    

In [ ]:
# Find the transform
mm = micrometry.Micrometry("locs4.hdf5",
                           min_size = 5.0,
                           max_size = 150.0,
                           max_neighbors = 20)

[best_ratio, best_transform] = mm.findTransform("locs5.hdf5", 1.0e-2)


In [ ]:
# Print the ratio and plot the best match.

print("Best ratio: {0:.1f}".format(best_ratio))
print("Best transform:")
print()
micrometry.prettyPrintTransform(best_transform)

micrometry.plotMatch(mm.getRefKDTree(),
                     mm.getOtherKDTree(),
                     best_transform)


Micrometry can find the correct transform even when most of the localizations are noise, but it can take a while as it will need more quads to compare.

### Localizations that don't match

In [ ]:
numpy.random.seed(3)

locs6 = {"x" : numpy.random.uniform(high = im_size, size = n_points),
         "y" : numpy.random.uniform(high = im_size, size = n_points)}

with saH5Py.SAH5Py("locs6.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs6, 0)

locs7 = {"x" : numpy.random.uniform(high = im_size, size = n_points),
         "y" : numpy.random.uniform(high = im_size, size = n_points)}

with saH5Py.SAH5Py("locs7.hdf5", is_existing = False, overwrite = True) as h5:
    h5.setMovieInformation(im_size, im_size, 1, "")
    h5.addLocalizations(locs7, 0)
    

In [ ]:
# Find the transform
mm = micrometry.Micrometry("locs6.hdf5",
                           min_size = 5.0,
                           max_size = 200.0,
                           max_neighbors = 20)

[best_ratio, best_transform] = mm.findTransform("locs7.hdf5", 1.0e-2)

In [ ]:
# Print the ratio and plot the best match.

print("Best ratio: {0:.2f}".format(best_ratio))
print("Best transform:")
print()
micrometry.prettyPrintTransform(best_transform)

micrometry.plotMatch(mm.getRefKDTree(),
                     mm.getOtherKDTree(),
                     best_transform)